In [7]:
using Laplacians
using DataStructures
include("../src/cutPageRank.jl")
include("../src/flowClustering.jl")
include("../src/cutHeuristics.jl")

dumb (generic function with 1 method)

In [ ]:
function condTest()
    a = chimera(2500)
    s = prn(a, [1,2,3], 0.2, 5);
    conds = compConductance(a, s)
    if length(s) < 30
        return -1,0,0,0
    end
    
    minEpsSigma = getVolume(a, s) / getVolume(a, setdiff(collect(1:max(a.n, a.m)), s))
    cut, flow = localImprove(a, s, epsSigma = minEpsSigma)
    condcut = compConductance(a, cut)
    
    heur_refcut = refineCut(a, s)
    condref = compConductance(a, heur_refcut)
    
    heur_dumb = dumb(a, s)
    conddumb = compConductance(a, heur_dumb)
    
    println(conds, " with ", length(s), " vertices")
    println(condcut, " with ", length(cut), " vertices. improvement = ", (conds - condcut) / conds * 100, " <- flow")
    println(condref, " with ", length(heur_refcut), " vertices. improvement = ", (conds - condref) / conds * 100)
    println(conddumb, " with ", length(heur_dumb), " vertices. improvement = ", (conds - conddumb) / conds * 100)
    println("*********")
    
    
    return conds, condcut, condref, conddumb
end

initial = []
flowbased = []
heurbased = []
dumbbased = []

for i in 1:300
    println("Test ", i)
    
    x,y,z,t = condTest()
    if x == -1
        continue
    end
    
    push!(initial, x)
    push!(flowbased, y)
    push!(heurbased, z)
    push!(dumbbased, t)
end

Test 1
0.19137466307277629 with 86 vertices
0.0064516129032258064 with 155 vertices. improvement = 96.62880508859611 <- flow
0.10222222222222223 with 109 vertices. improvement = 46.58528951486698
0.07151370679380215 with 205 vertices. improvement = 62.631570111970994
*********
Test 2
Test 3

In [4]:
println(mean(initial), " ", median(initial))
println(mean(flowbased), " ", median(flowbased))
println(mean(heurbased), " ", median(heurbased))
println(mean(dumbbased), " ", median(dumbbased))

0.17779807409138323 0.1891891891891892
0.0625237525362409 0.03851851851851852
0.15191694488648882 0.16543843977966355
0.11534092824471844 0.12689914441275602


In [ ]:
function condTest()
    a = chimera(100);
    s = prn(a, [1,2,3], 0.2, 5);
    if length(s) == 0
        return
    end
    
    # construct the vector of test EpsSigma values
    minEpsSigma = getVolume(a, s) / getVolume(a, setdiff(collect(1:max(a.n, a.m)), s))
    aux = [minEpsSigma + 0.0001, minEpsSigma + 0.02, 0.05, 0.1, 0.2, 0.4, 1.0, 2.0, 4.0, 10.0, 30.0]
    epsSigma = []
    for eS in aux
        if eS > minEpsSigma
            push!(epsSigma, eS)
        end
    end
    sort!(epsSigma)
    
    refcut = refineCut(a, s)
    condref = compConductance(a, refcut)
    
    # do the testing
    for eS in epsSigma
        cut, flow = localImprove(a, s, epsSigma = eS)
       
        conds = compConductance(a, s)
        condcut = compConductance(a, cut)
        
        if eS == minEpsSigma + 0.0001
#             println(conds, " ", condcut, " improvement(epsig = ", eS,") = ", 100 * (conds - condcut) / conds, "%", " initial set size = ", length(s), " improved set size = ", length(cut))
#             println(conds, " ", condref, " improvement = ", 100 * (conds - condref) / conds, "%", " initial set size = ", length(s), " improved set size = ", length(refcut))
#             println("!!!!")
        end
        if (conds < condcut)
            for iter in 1:100
                println("@@@@@@@@@@@@@@@@@@@@")
            end
            println(conds, " ", condcut, " improvement(epsig = ", eS,") = ", 100 * (conds - condcut) / conds, "%", " initial set size = ", length(s), " improved set size = ", length(cut))
            println(a)
            println(s)
        end
    end
end

for i in 1:10000
    if i % 100 == 1
        println("Test ", i)
    end
#     println("Test ", i)
    condTest()
#     println("*************************")
end

In [ ]:
a = chimera(500);
@time x, y = spectralCoords(a);

In [ ]:
include("../src/cutPageRank.jl")
@time s = prn(a, [100,101,102], 0.2, 4);
plotGraph(a,x,y,[0,0,1])
plotGraph(a[s,s],x[s],y[s],[1,0,0],setaxis=false)

In [ ]:
include("../src/flowClustering.jl")
epsSigma = 10.0

@time cut, flow = localImprove(a, s, epsSigma)

println(compConductance(a, s))
println(compConductance(a, cut))

plotGraph(a,x,y,[0,0,1])
plotGraph(a[cut,cut],x[cut],y[cut],[1,0,0],setaxis=false)